In [13]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

path: String = "/home/jovyan/source/load-ivy.sc"

In [2]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test
import dotvisualizer._

import chisel3._

import chisel3.util._

import chisel3.tester._

import chisel3.tester.RawTester.test

import dotvisualizer._

# 1.2 Introduction to Chisel

Chisel (Constructing Hardware In a Scala Embedded Language) is simply a set of predefined special
class definitions, objects as well as usage conventions within Scala.A Chisel program  is actually
a Scala program, which constructs the hardware modules when compiled.


## 1.2.1 Chisel Datatypes


In Chisel, datatypes specify the type of values held in state elements (register or memory ) or flowing
on wires. The datatypes in Chisel are different from the ones in Scala. In some cases, we may need
to cast (typecast) between Scala and Chisel types. Furthermore, casting between Chisel types may
also be required.
Unsigned and signed integers are represented by the keywords UInt and SInt respectively

In [3]:
val X1 = 21. S(32.W) //defines a signed integer literal with a value of 23 and a width of 32 bits.The W method call on the UInt type specifies the bit width of the integer.

X1: SInt = SInt<32>(21)

In [4]:
0.B
true
true.B
val a: Bool = true.B

6
6.U
6.U(8.W)
val myUInt = 4.U
val myUint8 = 4.U(8.W)

-2
-2.S

res3_0: Bool = Bool(false)
res3_1: Boolean = true
res3_2: Bool = Bool(true)
a: Bool = Bool(true)
res3_4: Int = 6
res3_5: UInt = UInt<3>(6)
res3_6: UInt = UInt<8>(6)
myUInt: UInt = UInt<3>(4)
myUint8: UInt = UInt<8>(4)
res3_9: Int = -2
res3_10: SInt = SInt<2>(-2)

## 1.2.2 Counter Class 


Below we implement the Counter class using Chisel library to generate the corresponding
hardware module.


In [8]:
import chisel3 . _
class Counter ( counterBits : UInt ) extends Module {
    val io = IO (new Bundle {
        val result = Output ( Bool () )
    })
val max = (1. U << counterBits ) - 1. U
val count = RegInit (0. U (16. W ) )
when ( count === max ) {
    count := 0. U
} .otherwise {
    count := count + 1. U
}
io . result := count (15. U )
println ( s" counter created with max value $max ")
}


import chisel3 . _

defined class Counter

# 1.3 Optimization of Signals and Parametrized Hardware Generation

variable ‘y1’ is first initialized with unsigned integer 23 then it is converted to signed
number 9 which is 2’s complement of 23. In the FIRRTL3 generated verilog 9 will be subtracted from
the input io.x.


In [9]:
import chisel3 . _
class AdderWithOffset extends Module {
val io = IO (new Bundle {
val x = Input ( SInt (16. W ) )
val y = Input ( UInt (16. W ) )
val z = Output ( UInt (16. W ) )
})
// Initialized as UInt and casted to SInt
val y1 = (23. U ) . asSInt
val in1 = io . x + y1
io . z := in1 . asUInt + io . y // Typecast SInt to UInt
}
println (( new chisel3 . stage . ChiselStage ) . emitVerilog (new AdderWithOffset ) )

Elaborating design...
Done elaborating.
module AdderWithOffset(
  input         clock,
  input         reset,
  input  [15:0] io_x,
  input  [15:0] io_y,
  output [15:0] io_z
);
  wire [15:0] _T_2 = $signed(io_x) - 16'sh9; // @[cmd8.sc 11:17]
  assign io_z = _T_2 + io_y; // @[cmd8.sc 11:24]
endmodule



import chisel3 . _

defined class AdderWithOffset

## 1.3.1 Parametrized Hardware Generation

Functions are defined using keyword def. Similar to the class declaration, it also has a name followed
by parameters list. Every function has a return type. If the return type is not defined during
declaration then the last line of the function block will be the returned value and its type will be
inferred.

Based on given parameters, Chisel code will be configured accordingly. For instance, 
’size’ and ’maxValue’ parameters will configure the counter hardware for bitwidth and reload value,
respectively. For size = 8 and maxValue = 255 a counter register with width ‘8’ will be initialized
with zero value and parameter maxValue will set the maximum value at which counter will restart the
count. One bit output is derived from the most significant bit (MSB) of the counter register, which
can be used as a clock divisor.


In [11]:
import chisel3 . _
class Counter ( size : Int , maxValue : UInt ) extends Module {
    val io = IO (new Bundle {
        val result = Output ( Bool () )
    })
// 'genCounter ' with counter size 'n'
    def genCounter ( n : Int , max : UInt ) = {
        val count = RegInit (0. U ( n . W ) )
        when ( count === max ) {
            count := 0. U
        } .otherwise {
            count := count + 1. U
    }
    count
    }
    val counter1 = genCounter ( size , maxValue )
    io . result := counter1 ( size -1)
}    

import chisel3 . _

defined class Counter

In [12]:

println (( new chisel3 . stage . ChiselStage ) . emitVerilog (new Counter (8 , 255. U ) ) )


Elaborating design...
Done elaborating.
module Counter(
  input   clock,
  input   reset,
  output  io_result
);
`ifdef RANDOMIZE_REG_INIT
  reg [31:0] _RAND_0;
`endif // RANDOMIZE_REG_INIT
  reg [7:0] counter1; // @[cmd10.sc 8:29]
  wire [7:0] _T_2 = counter1 + 8'h1; // @[cmd10.sc 12:28]
  assign io_result = counter1[7]; // @[cmd10.sc 17:29]
  always @(posedge clock) begin
    if (reset) begin // @[cmd10.sc 8:29]
      counter1 <= 8'h0; // @[cmd10.sc 8:29]
    end else if (counter1 == 8'hff) begin // @[cmd10.sc 9:32]
      counter1 <= 8'h0; // @[cmd10.sc 10:19]
    end else begin
      counter1 <= _T_2; // @[cmd10.sc 12:19]
    end
  end
// Register and memory initialization
`ifdef RANDOMIZE_GARBAGE_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_INVALID_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_REG_INIT
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_MEM_INIT
`define RANDOMIZE
`endif
`ifndef RANDOM
`define RANDOM $random
`endif
`ifdef RANDOMIZE_MEM_INIT
  integer initvar;
`endif
`i

# Bonus work

## chisel operators

In [5]:
val x1 = 23. S (32. W )
val x2 = 22. S (32. W )
val x3 =  Input( Bool () )

x1: SInt = SInt<32>(23)
x2: SInt = SInt<32>(22)
x3: Bool = Bool

## First chisel module

In [6]:
class MyXOR extends Module {
    val io = IO(new Bundle {
        val a = Input(Bool())
        val b = Input(Bool())
        val c = Output(Bool())
    
    })
    io.c := io.a ^ io.b 
}

defined class MyXOR

In [7]:
class Passthrough extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(4.W))
    val out = Output(UInt(4.W))
  })
  io.out := io.in
}

defined class Passthrough

## Looking At Generated Design

In [4]:
println(getVerilog(new MyXOR))

Elaborating design...
Done elaborating.
module MyXOR(
  input   clock,
  input   reset,
  input   io_a,
  input   io_b,
  output  io_c
);
  assign io_c = io_a ^ io_b; // @[cmd2.sc 8:18]
endmodule



In [6]:
println(getVerilog(new Passthrough))

Elaborating design...
Done elaborating.
module Passthrough(
  input        clock,
  input        reset,
  input  [3:0] io_in,
  output [3:0] io_out
);
  assign io_out = io_in; // @[cmd4.sc 6:10]
endmodule



## Looking at Generated Visualize Design

In [5]:
visualize(() => new MyXOR)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<?xml-stylesheet href="styles.css" type="text/css"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.40.1 (20161225.0304)
 -->
<!-- Title: MyXOR Pages: 1 -->
 
 
 MyXOR 
 
 
 cluster_MyXOR 
 
 MyXOR 
 
<!-- cluster_MyXOR_clock -->
 
 cluster_MyXOR_clock 
 
 clock 
 
<!-- cluster_MyXOR_reset -->
 
 cluster_MyXOR_reset 
 
 reset 
 
<!-- cluster_MyXOR_io_a -->
 
 cluster_MyXOR_io_a 
 
 io_a 
 
<!-- op_xor_1 -->
 
 op_xor_1 
 
 
 ● 
 
 xor 
 
 ● 
 
 ● 
 
<!-- cluster_MyXOR_io_a->op_xor_1 -->
 
 cluster_MyXOR_io_a->op_xor_1:in1 
 
 
 
<!-- cluster_MyXOR_io_b -->
 
 cluster_MyXOR_io_b 
 
 io_b 
 
<!-- cluster_MyXOR_io_b->op_xor_1 -->
 
 cluster_MyXOR_io_b->op_xor_1:in2 
 
 
 
<!-- cluster_MyXOR_io_c -->
 
 cluster_MyXOR_io_c 
 
 io_c 
 
<!-- op_xor_1->cluster_MyXOR_io_c -->
 
 op_xor_1:out->cluster_MyXOR_io_c

## Brief ChiselTest Intro

### poke value of wire 
### peek read value of wire 
### expect read value and compare (assert)

In [8]:
test(new MyXOR()){ x =>
    
    x.io.a.poke(0.B)
    x.io.b.poke(0.B)
    x.io.c.expect(0.B) // 0 ^ 0
    
    x.io.a.poke(0.B)
    x.io.b.poke(1.B)
    x.io.c.expect(1.B) // 0 ^ 1
    
    x.io.a.poke(1.B)
    x.io.b.poke(0.B)
    x.io.c.expect(1.B) // 1 ^ 0
    
    x.io.a.poke(1.B)
    x.io.b.poke(1.B)
    x.io.c.expect(0.B) // 1 ^ 1
}

Elaborating design...
Done elaborating.
test MyXOR Success: 0 tests passed in 2 cycles in 0.211347 seconds 9.46 Hz
